In [2]:
import datetime as dt
import pandas as pd

In [3]:
df = pd.read_csv("weights_account_to_device.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)
df.columns = ["AccountName", "DeviceInfo", "Weight", "TransactionID", "isFraud", "TimeStep", "AmountTransferred", "ServiceType"]


x = lambda a : dt.datetime.fromtimestamp(a)
df.TimeStep = pd.to_datetime(df.TimeStep.apply(x), format="%Y-%m-%d %H:%M:%S")

In [11]:
accounts_number_of_devices = pd.read_csv("accounts_and_number_of_devices.csv")

Categories of customers: <br>
1-5 devices: individuals (I) <br>
6-10: small companies (S) <br>
11-50: medium companies (M) <br>
\> 50: big companies (B)

In [14]:
customer_profiles = []
for i in range(len(accounts_number_of_devices)):
    if accounts_number_of_devices.degree.iloc[i] <= 5:
        customer_profiles.append("I")
    elif (accounts_number_of_devices.degree.iloc[i] > 5) & (accounts_number_of_devices.degree.iloc[i] < 11):
        customer_profiles.append("S")
    elif (accounts_number_of_devices.degree.iloc[i] >= 11) & (accounts_number_of_devices.degree.iloc[i] < 51):
        customer_profiles.append("M")
    else:
        customer_profiles.append("B")
        
accounts_number_of_devices = accounts_number_of_devices.assign(profile=customer_profiles)

In [15]:
accounts_number_of_devices["profile"].value_counts()

I    7093
S     413
M     200
B      44
Name: profile, dtype: int64

### How much fraud among different profiles

The table below shows fraud amoing transactions by different types of customers. The customer type is determined from the number of devices they accessed to make transactions.

In [21]:
pd.merge(accounts_number_of_devices, df, on="AccountName").groupby("profile")["isFraud"].value_counts()

profile  isFraud
B        0          42348
         1           4135
I        0          26302
         1           1232
M        0          28115
         1           2182
S        0          12286
         1            786
Name: isFraud, dtype: int64